<a href="https://colab.research.google.com/github/Jawad1347/MachineLearning/blob/main/Fine_Tuning_GPT_2_on_a_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning GPT-2 on a Custom Dataset

## Setup

Make sure you enable a GPU or TPU in the runtime  
Runtime -> Change Runtime -> Hardware Accelerator 

Install helper library for fine-tuning.

In [ ]:
!pip install -q gpt-2-simple

Imports

In [ ]:
# Colab pre-installs many common libraries including TensorFlow.
# Use Colab magic command to tell Colab to not use TensorFlow 2.0.
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

Download GPT-2 models.  
When fine-tuning on a single GPU, only the 124M and 335M size models can be used.

In [ ]:
#gpt2.download_gpt2(model_name="124M")
gpt2.download_gpt2(model_name="355M")


Set variables

In [ ]:
file_name = "my_custom_dataset.txt"
run_name = 'fine_tuning_run_1'
model_size = '355M'

If your custom data is stored in your G-Drive, mount your drive and you can copy the data to Colab with the code below.  
Alternatively, you can upload your dataset directly to Colab using the Colab "Files" menu on the left (not the "File" menu above).  
Training examples in the dataset file should be separated with a blank line.

In [ ]:
gpt2.mount_gdrive()

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Perform fine-tuning

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_size,
              steps=200,
              restore_from='fresh',
              run_name = run_name,
              print_every=10,
              sample_every=50,
              save_every=50)
             # , learning_rate=.00003)

In [ ]:
# If G-Drive is mounted, save the fine-tuned model to the drive.
gpt2.copy_checkpoint_to_gdrive(run_name)

End of training

## Explore results

In [ ]:
print(run_name)

In [ ]:
 # Copy the model from G-Drive if it wasn't trained in this Colab session.
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

In [ ]:
# Must reset the graph if training was done in this Colab session.
import tensorflow as tf
tf.reset_default_graph()

In [ ]:
# Load the model.
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Unconditional generation

In [ ]:
gpt2.generate(sess, run_name=run_name, temperature=.7, length=100, prefix=None, top_k=40, nsamples=10)

Conditional generation (give the model an input prompt)

In [ ]:
input_prompt = "Today's weather is"

In [ ]:
gpt2.generate(sess, run_name=run_name, temperature=.7, length=100, prefix=input_prompt, top_k=40, nsamples=10)

## Copy model to Google Cloud Storage  
We have already copied the model to your G-Drive.  
This code will copy the model to your Google Cloud Platform Cloud Storage account (not free), where it can be used in data science deployment pipelines.


In [ ]:
project_id = 'your_gcp_project_id'
bucket_name = 'your_gcp_bucket_name'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project {project_id}

In [ ]:
# Need to zip just the inner folder.
# Currently, the name of the folder inside the zip needs to be `run1`.
!cd checkpoint; zip -r ../your_model_name.zip .

In [ ]:
!gsutil -m cp your_model_name.zip gs://{bucket_name}